In [1]:
import sqlite3
import pandas as pd
import sys
import time

In [2]:
def age_at_election(dob, election_date):
    '''Given an election date and a date of birth, return the person's age 
    in years on election day. Return years as floor of years - ie, if someone is 70 years old
    and 360 days, return 70. '''
    try:
        return relativedelta(pd.to_datetime(election_date), pd.to_datetime(dob, format="%Y%m%d")).years
    except:
        return float('nan')

def young_at_election(series, election_date):
    '''Given an election date and a date of birth pandas Series, return 
    a Series whether or not someone was 24 or under on election day.
    Return years as floor of years - ie, if someone is 70 years old
    and 360 days, return 70. '''
    return pd.to_datetime(series, format = "%Y%m%d", errors = "coerce") + pd.DateOffset(years=24) >= election_date

def find_election(array, year, e_type):
    if array is None:
        return None
    
    if e_type.lower() == "general":
        e_type = "GE"
    elif e_type.lower() == "primary":
        e_type = "PR"
    else:
        return None
    
    year = str(year)
    
    # Some elections might use YY instead of YYYY
    # TODO: unsolved
    # New bug: some elections recorded as %m%d%Y
    # So I would also return true for something like '061416' -> June, 14 2016 election
    # But that wouldn't have "GE" in it usually
    # An exception would be something like "GENEVA election 2016"
    # But that's being captured errorenously anyway --> TODO!
    # For now, I'm going to do that
    # This is a quick and cheap data check anyway
    
    for election in array:
        if year in election and e_type in election:
            return True
    return False

In [3]:
# Loading bar from: https://gist.github.com/vladignatyev/06860ec2040cb497f0f3
def progress(count, total, status=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status))
    sys.stdout.flush() # As suggested by Rom Ruben (see: http://stackoverflow.com/questions/3173320/text-progress-bar-in-the-console/27871113#comment50529068_27871113)

In [4]:
read_cols = ["City", "Zip5","DOB","Gender","Party","County_Code",\
        "CD","State_Senate",\
        "State_Assembly","Reg_Date","Status","StateID", "Voter_Hx"]

In [5]:
with sqlite3.connect("../data/govt990.db") as db:
    query = """SELECT COUNT(*)
    FROM nys_young_voters;"""
    nrows = pd.read_sql(query, db).iloc[0,0]
nrows

2770204

In [6]:
with sqlite3.connect("../data/govt990.db") as db:
    query = """SELECT COUNT(DISTINCT(stateid))
    FROM nys_young_voters;"""
    nvoters = pd.read_sql(query, db).iloc[0,0]
nvoters

2609291

In [7]:
import pdb

In [8]:
# I want to see how many people under 24 the data said voted in the 2014 midterm elections.
# After, I will look at how many people under 24 the data said voted in the 2010 midterms. 
# I will read in all of the data, and then get the count of rows which were <= 24 at 
# the time of the given election AND voted in the general election that year. 
def my_fnct(debug = False):
    elections = ["11/4/14", "11/2/10"] # midterm dates
    voted2014 = 0 # number of young people who voted, init at 0
    voted2010 = 0 
    count = 0 # to keep track of progress
    myChunkSize = int(1e6) # chunksize

    start =time.time()

    with sqlite3.connect("../data/govt990.db") as db:
        query = """SELECT *
        FROM nys_young_voters;"""
        reader = pd.read_sql(query, db, chunksize = myChunkSize)
        for chunk in reader:
            count += len(chunk)
            chunk["Young2010"] = young_at_election(chunk["DOB"], election_date = elections[1])
            chunk["Voted2010"] = chunk["Voter_Hx"].str.upper().str.split(";").apply(find_election, year = 2010, e_type = "General")

            voted2010 += len(chunk[(chunk["Young2010"] == True) & (chunk["Voted2010"] == True)])

            chunk["Young2014"] = young_at_election(chunk["DOB"], election_date = elections[0])
            chunk["Voted2014"] = chunk["Voter_Hx"].str.upper().str.split(";").apply(find_election, year = 2014, e_type = "General")
            voted2014 += len(chunk[(chunk["Young2014"] == True) & (chunk["Voted2014"] == True)])

            if debug == True:
                pdb.set_trace()
            # voted204 += chunk["Voted2014"].sum()

            progress(count, nrows)
#             passed = time.time() - start
#             if passed > 120:
#                 sys.exit()
    return({'voted14' :voted2014, 'voted10': voted2010})

Bugs:
- It looks like the 2010 records have duplicate records (and a lot of them!)
- The 2014 records are undercounting voters, and I do not know why

In [9]:
res = my_fnct(debug=False)

In [10]:
res

{'voted10': 141273, 'voted14': 93726}

In [11]:
voted2014 = 205047

In [12]:
chunk[not chunk["Young2010"] & chunk["Voted2010"]]

NameError: name 'chunk' is not defined

In [ ]:
actualVote = int(258e3) # from https://www.census.gov/data/tables/time-series/demo/voting-and-registration/p20-577.html
actualVote - voted2014

That is a huge discrepency. It is part becasue the Census uses self-reporting.